# Experiment 003: Comprehensive Ensemble

Compare all available pre-optimized sources and pick the best configuration for each N value.

In [1]:
import pandas as pd
import numpy as np
import glob
import os
from numba import njit
import math

# Tree geometry
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    return np.array([float(str(v).replace("s","")) for v in a], np.float64)

tx, ty = make_polygon_template()
print("Functions defined")

Functions defined


In [2]:
# Find all CSV files in preoptimized folder
preopt_base = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized'

csv_files = []
for root, dirs, files in os.walk(preopt_base):
    for f in files:
        if f.endswith('.csv'):
            csv_files.append(os.path.join(root, f))

print(f"Found {len(csv_files)} CSV files:")
for f in csv_files:
    print(f"  {f}")

Found 30 CSV files:
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/submission.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/best_ensemble.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/72.49.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/71.97.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/telegram_extracted/72.49.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/telegram_extracted/71.97.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/submission_JKoT4.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/New_Tree_144_196.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimi

In [3]:
# Load and compare all sources
best = {n: {"score": 1e300, "data": None, "src": None} for n in range(1, 201)}

for fp in csv_files:
    try:
        df = pd.read_csv(fp)
    except Exception as e:
        print(f"Error loading {fp}: {e}")
        continue
    
    if not {"id", "x", "y", "deg"}.issubset(df.columns):
        print(f"Skipping {fp} - missing columns")
        continue
    
    df = df.copy()
    df["N"] = df["id"].astype(str).str.split("_").str[0].astype(int)
    
    for n, g in df.groupby("N"):
        if n < 1 or n > 200:
            continue
        xs = strip(g["x"].to_numpy())
        ys = strip(g["y"].to_numpy())
        ds = strip(g["deg"].to_numpy())
        sc = score_group(xs, ys, ds, tx, ty)
        if sc < best[n]["score"]:
            best[n]["score"] = float(sc)
            best[n]["data"] = g.drop(columns=["N"]).copy()
            best[n]["src"] = os.path.basename(fp)

print("\nDone scanning all sources")


Done scanning all sources


In [4]:
# Show which sources are best for each N
source_counts = {}
for n in range(1, 201):
    src = best[n]["src"]
    if src:
        source_counts[src] = source_counts.get(src, 0) + 1

print("Source distribution:")
for src, count in sorted(source_counts.items(), key=lambda x: -x[1]):
    print(f"  {src}: {count} N values")

Source distribution:
  ensemble.csv: 200 N values


In [5]:
# Calculate total score
total_score = sum(best[n]["score"] for n in range(1, 201))
print(f"\nTotal ensemble score: {total_score:.6f}")
print(f"Baseline score: 70.676102")
print(f"Improvement: {70.676102 - total_score:.6f}")


Total ensemble score: 70.676102
Baseline score: 70.676102
Improvement: -0.000000


In [6]:
# Show top 20 contributors to score
per_n = [(n, best[n]["score"], best[n]["src"]) for n in range(1, 201)]
per_n.sort(key=lambda x: -x[1])

print("\nTop 20 score contributors:")
for n, score, src in per_n[:20]:
    print(f"  N={n:3d}: {score:.6f} from {src}")


Top 20 score contributors:
  N=  1: 0.661250 from ensemble.csv
  N=  2: 0.450779 from ensemble.csv
  N=  3: 0.434745 from ensemble.csv
  N=  5: 0.416850 from ensemble.csv
  N=  4: 0.416545 from ensemble.csv
  N=  7: 0.399897 from ensemble.csv
  N=  6: 0.399610 from ensemble.csv
  N=  9: 0.387415 from ensemble.csv
  N=  8: 0.385407 from ensemble.csv
  N= 15: 0.379203 from ensemble.csv
  N= 10: 0.376630 from ensemble.csv
  N= 21: 0.376451 from ensemble.csv
  N= 20: 0.376057 from ensemble.csv
  N= 11: 0.375736 from ensemble.csv
  N= 22: 0.375258 from ensemble.csv
  N= 16: 0.374128 from ensemble.csv
  N= 26: 0.373997 from ensemble.csv
  N= 12: 0.372724 from ensemble.csv
  N= 13: 0.372323 from ensemble.csv
  N= 25: 0.372144 from ensemble.csv


In [7]:
# Build the ensemble submission
rows = []
for n in range(1, 201):
    entry = best[n]
    if entry["data"] is not None:
        rows.append(entry["data"])

out = pd.concat(rows, ignore_index=True)
out["sn"] = out["id"].str.split("_").str[0].astype(int)
out["si"] = out["id"].str.split("_").str[1].astype(int)
out = out.sort_values(["sn", "si"]).drop(columns=["sn", "si"])
out = out[["id", "x", "y", "deg"]]

print(f"Ensemble has {len(out)} rows")
print(out.head(10))

Ensemble has 20100 rows
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   
5  003_2      s0.641714640229075      s1.180458566613381   
6  004_0  s-0.324747789589372171   s0.132109978088185392   
7  004_1   s0.315354346242637695   s0.132109978063475492   
8  004_2   s0.324747789592379210  s-0.732109978069475531   
9  004_3  s-0.315354348134818330  s-0.732109978094185987   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  
5      s155.13405193710082  
6  s156.370622145636389178  
7  s156.370622269264089255  
8  s336.370622269264003990  
9  s336.37062214563644602

In [ ]:
# Save ensemble
out.to_csv('/home/code/experiments/003_ensemble/submission.csv', index=False)
out.to_csv('/home/submission/submission.csv', index=False)
print("Saved ensemble submission")

# Verify score
df_verify = pd.read_csv('/home/submission/submission.csv')
df_verify["N"] = df_verify["id"].astype(str).str.split("_").str[0].astype(int)

verify_total = 0
for n, g in df_verify.groupby("N"):
    xs = strip(g["x"].to_numpy())
    ys = strip(g["y"].to_numpy())
    ds = strip(g["deg"].to_numpy())
    sc = score_group(xs, ys, ds, tx, ty)
    verify_total += sc

print(f"\nVerified total score: {verify_total:.6f}")